In [18]:
import pyreadstat
import pandas as pd

# .sav 파일을 읽을 때 df와 meta를 모두 받습니다.
df, meta = pyreadstat.read_sav("data_sy/2024 외래관광객조사_Data.sav", encoding="cp949")

# --- 메타데이터 확인 ---

# 1. 변수명과 실제 질문 내용(레이블) 확인
# 예: {'Q1': '여행 만족도', 'Q2': '재방문 의향'}
print("### 변수 이름 -> 변수 레이블 매핑 ###")
#print(meta.column_names_to_labels)
variable_map = meta.column_names_to_labels

# 2. 값과 값의 의미(레이블) 확인 (가장 중요한 부분!)
# 예: {'성별': {1.0: '남자', 2.0: '여자'}, '학력': {1.0: '고졸', 2.0: '대졸', ...}}
#print("\n### 값 -> 값 레이블 매핑 ###")
#print(meta.variable_value_labels)
value_map = meta.variable_value_labels

### 변수 이름 -> 변수 레이블 매핑 ###


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16216 entries, 0 to 16215
Columns: 402 entries, pnid to weight
dtypes: float64(402)
memory usage: 49.7 MB


In [20]:
value_labels_map = meta.variable_value_labels
df_processed = df.copy()
# 2. 매핑 정보가 있는 모든 변수에 대해 값 변환 실행
for var_name, mapping_dict in value_labels_map.items():
    # 2-1. 해당 변수명(컬럼)이 df에 있는지 확인
    if var_name in df_processed.columns:
        # 2-2. .map()을 사용하여 숫자 코드를 문자열 값으로 변환
        df_processed[var_name] = df_processed[var_name].map(mapping_dict)

In [ ]:
import re

# 지울 변수
columns_to_drop = []
# MVIT, XRVIT, RVIT (중복 with D_NUM)
columns_to_drop.extend(['MVIT', 'XRVIT', 'RVIT'])
# TYP (중복 with D_GUB)
columns_to_drop.append('TYP')
columns_to_drop.extend([col for col in df.columns if col.startswith(('weight'))])
columns_to_drop.extend([
    "MDAY개별대체61", "MDAY에어대체61", "RDAY개별대체61", "RDAY에어대체61", "RDAY단체대체61",
    "MDAY개별대체_개별국제교통비제외61", "MDAY에어대체_개별국제교통비제외61", "MDAY단체대체_개별국제교통비제외61",
    "RDAY개별대체_개별국제교통비제외61", "RDAY에어대체_개별국제교통비제외61", 
    'MDAY전체TOT_RAW61', 'RDAY전체TOT_RAW61', 'MDAY전체_개별국제교통비제외61', 'RDAY전체_개별국제교통비제외61'

])
columns_to_drop.extend([
    'MQ7_1' ,'MQ7_1제외' ,'MQ7_2' ,'MQ7_1$15제외', 'MQ7_1제외$15제외',
    'RQ7_1제외' ,'RQ7_2', 'RQ7_1$15제외', 'RQ7_1제외$15제외'
])
columns_to_drop.extend([
    'KWON1', 'KWON2', 'KWON3', 'KWON4',
    'KWON5', 'KWON6', 'KWON7', 'KWON8'
])
columns_to_drop.extend([
    'KWONB1', 'KWONB2'
])
columns_to_drop.extend([
    'M박HAP', 'M일HAP', 'M일HAP_61'
])
columns_to_drop.extend([
    '서울박TOT', '경기박TOT', '인천박TOT', '강원박TOT', '대전박TOT', '충북박TOT',
    '충남박TOT', '세종박TOT', '경북박TOT', '경남박TOT', '대구박TOT', '울산박TOT',
    '부산박TOT', '광주박TOT', '전북박TOT', '전남박TOT', '제주박TOT'
])
columns_to_drop.extend([
    '서울박60', '경기박60', '인천박60', '강원박60', '대전박60', '충북박60',
    '충남박60', '세종박60', '경북박60', '경남박60', '대구박60', '울산박60',
    '부산박60', '광주박60', '전북박60', '전남박60', '제주박60'
])
columns_to_drop.extend([
    '서울일61', '경기일61', '인천일61', '강원일61', '대전일61', '충북일61',
    '충남일61', '세종일61', '경북일61', '경남일61', '대구일61', '울산일61',
    '부산일61', '광주일61', '전북일61', '전남일61', '제주일61'
])
columns_to_drop.extend([f'WQ9_5a{str(i).zfill(2)}' for i in range(1, 18)])
columns_to_drop.extend([
    '호텔숙박', '모텔숙박', '콘도숙박', '게스숙박', '민박숙박', '학교숙박', '친척숙박', '기타숙박',
    '호텔숙박60', '모텔숙박60', '콘도숙박60', '게스숙박60', '민박숙박60', '학교숙박60', '친척숙박60', '기타숙박60'
])
columns_to_drop.extend([
    '총액1인TOT2', '총액1인TOT_개별국제교통비제외2'
])
columns_to_drop.extend([
    '여행사1인대체', '가이드1인대체', '숙박비1인대체', '음식점1인대체', '식음료1인대체', '국제한국1인대체', '국제국외1인대체', '국제수상1인대체',
    '한국한국1인대체', '한국국외1인대체', '한국수상1인대체', '한국철도1인대체', '한국도로1인대체', '대여서1인대체', '유류비1인대체',
    '문화서1인대체', '오락및1인대체', '쇼핑비1인대체', '데이터1인대체', '치료및1인대체', '미용서1인대체', '기타비1인대체', '단기투어상품1인대체'
])

columns_to_drop = list(set(columns_to_drop))
df_processed.drop(columns=columns_to_drop, inplace=True)

new_rename_map = {
    old_name: new_name
    for old_name, new_name in variable_map.items()
    if old_name in df_processed.columns
}



df_processed = df_processed.rename(columns=new_rename_map)

cols_to_clean = [
    '▩ 월              별 ▩', ' ▩ 분      기      별 ▩', ' ▩ 국      가      별 ▩', 
    ' ▩ 국  가  별(중동) ▩', ' ▩ 성              별 ▩', ' ▩ 연      령      별 ▩', 
    ' ▩ 방  한  목  적  별 ▩', ' ▩ 방  한  횟  수  별 ▩', ' ▩ 여  행  형  태  별 ▩'
]

# { '기존 이름': '새 이름' } 형태의 딕셔너리 생성
rename_dict = {
    col: re.sub(r'[▩\s]', '', col) 
    for col in cols_to_clean 
    if col in df_processed.columns # 데이터프레임에 실제 존재하는 컬럼만 대상으로 함
}

# rename 메서드로 컬럼명 변경
df_processed.rename(columns=rename_dict, inplace=True)

# 결과 확인
print(df_processed.columns)

cols_to_clean_values = [
    '월별', '분기별', '국가별', '국가별(중동)', '성별', '연령별', 
    '방한목적별', '방한횟수별', '여행형태별'
]

# 2. 각 컬럼을 순회하며 값 내부의 모든 공백 제거
for col in cols_to_clean_values:
    # 해당 컬럼이 존재하고, 데이터 타입이 문자열일 경우에만 실행
    if col in df_processed.columns and pd.api.types.is_string_dtype(df_processed[col]):
        df_processed[col] = df_processed[col].str.replace(r'\s', '', regex=True)



Index(['아이디', '문1. 주요 방한 목적', '문1-1. 한국여행 관심 계기(1순위)', '문1-1. 한국여행 관심 계기(2순위)',
       '문1-1. 한국여행 관심 계기(3순위)', '문2. 방문을 고려한 아시아 국가(1순위)',
       '문2. 방문을 고려한 아시아 국가(2순위)', '문2. 방문을 고려한 아시아 국가(3순위)',
       '문2. 방문을 고려한 아시아 국가(없음)', '문2-1. 해당 국가 관심 계기(1순위)',
       ...
       '문14. 타인 추천 의향', ' ▩ 월              별 ▩', '분기별', '국가별', '국가별(중동)', '성별',
       '연령별', '방한목적별', '방한횟수별', '여행형태별'],
      dtype='object', length=249)


In [22]:
survey_structure = {
    '기본정보': ['아이디'],
    
    '여행_준비': {
        '방한_목적_및_계기': {
            '주요_방한_목적': ['문1. 주요 방한 목적'],
            '관심_계기': ['문1-1. 한국여행 관심 계기(1순위)', '문1-1. 한국여행 관심 계기(2순위)', '문1-1. 한국여행 관심 계기(3순위)']
        },
        '방문국_고려': {
            '고려한_아시아_국가': ['문2. 방문을 고려한 아시아 국가(1순위)', '문2. 방문을 고려한 아시아 국가(2순위)', '문2. 방문을 고려한 아시아 국가(3순위)', '문2. 방문을 고려한 아시아 국가(없음)'],
            '타국가_관심_계기': ['문2-1. 해당 국가 관심 계기(1순위)', '문2-1. 해당 국가 관심 계기(2순위)', '문2-1. 해당 국가 관심 계기(3순위)'],
            '연계여행_정보': [ # 이 부분은 여러 항목이 중복되어 있어 정리가 필요해 보입니다.
                '문2-2. 한국만 방문(카테고리)', '문2-2. 한국 방문 직전 타 국가 방문(카테고리)', '문2-2. 한국 방문 직후 타 국가 방문(카테고리)', 
                '문2-2. 한국 방문 전후 타 국가 방문(카테고리)', '문2-2. 한국방문 직전 방문국가', '문2-2. 한국방문 직후 방문국가',
                '문2-2. 한국방문 직전 방문국가(숙박일수)', '문2-2. 한국방문 직후 방문국가(숙박일수)'
            ]
        },
        '고려사항': {
            '고려한_관광활동': ['문3-1. 고려한 관광활동(1순위)', '문3-1. 고려한 관광활동(2순위)', '문3-1. 고려한 관광활동(3순위)'],
            '고려한_관광인프라': ['문3-2. 고려한 관광인프라(1순위)', '문3-2. 고려한 관광인프라(2순위)', '문3-2. 고려한 관광인프라(3순위)']
        },
        '정보_수집': {
            '정보_수집_경로': ['문4. 여행 전 한국 관련 정보 수집 경로(1순위)', '문4. 여행 전 한국 관련 정보 수집 경로(2순위)', '문4. 여행 전 한국 관련 정보 수집 경로(3순위)', '문4. 여행 전 한국 관련 정보 수집 경로(없다)'],
            '주요_이용_사이트': ['문4-1. 주로 이용한 사이트(1순위)', '문4-1. 주로 이용한 사이트(2순위)', '문4-1. 주로 이용한 사이트(3순위)'],
            '부족했던_정보': ['문4-2. 부족했던 정보(1순위)', '문4-2. 부족했던 정보(2순위)', '문4-2. 부족했던 정보(3순위)', '문4-2. 부족했던 정보(없음)']
        },
        '예약_정보': {
            '예약_시기': ['문5. 왕복 항공권 및 여행상품 예약 시기'],
            '개별_예약_항목': [col for col in df_processed.columns if '문5-1. 개별 예약 항목' in col]
        }
    },

    '여행_중_행동': {
        '동반자_정보': {
            '동반자_유무': ['문7. 동반자 유무'],
            '동반자_유형': [col for col in df_processed.columns if '문7. 동반자 유형' in col],
            '동반자_수': [col for col in df_processed.columns if '문7-1. 동반자 수' in col]
        },
        '참여_활동': {
            '활동_목록': [col for col in df_processed.columns if '문8. 참여한 활동' in col],
            '만족한_활동': ['문8-1. 만족한 활동(1순위)', '문8-1. 만족한 활동(2순위)', '문8-1. 만족한 활동(3순위)'],
            '단기투어_이용': ['문6. 단기일일투어 상품 이용 여부', '문6. 단기일일투어 상품 이용 여부_기간', '문6. 단기일일투어 상품 이용 여부_기간(카테고리)']
        },
        '방문지_및_체류': {
            '가장_좋았던_곳': ['문9-1. 가장 좋았던 곳 - 1', '문9-1. 가장 좋았던 곳 - 2', '문9-1. 가장 좋았던 곳 - 3', '문9-1. 가장 좋았던 곳 - 4', '문9-1. 가장 좋았던 곳 - 5'],
            '방문_지역': [col for col in df_processed.columns if '문9-2. 방문 지역' in col],
            '방문_권역': [col for col in df_processed.columns if '문9-2. 방문권역' in col],
            '총_체류기간': ['문9-3. 총 숙박기간', '문9-3. 총 체재기간', '문9-3. 총 체재기간_61일 이상 결측', '문9-3. 총 체재기간(카테고리)'],
            '시도별_체류기간': [col for col in df_processed.columns if '문9-4. 시도별' in col],
            '숙박_정보': [col for col in df_processed.columns if '문9-5. ' in col]
        }
    },
    
    '여행_후_평가': {
        '지출_경비': {
            '총_지출경비': [col for col in df_processed.columns if '문10. ' in col],
            '항목별_지출경비': [col for col in df_processed.columns if '문10-1. ' in col],
            '쇼핑_세부사항': [col for col in df_processed.columns if '문10-2. ' in col or '문10-3. ' in col]
        },
        '만족도_및_의향': {
            '전반적_만족도': ['문11. 전반적 만족도'],
            '항목별_만족도': [col for col in df_processed.columns if '문12. 항목별 만족도' in col],
            '재방문_의향': ['문13. 재방문 의사'],
            '타인_추천_의향': ['문14. 타인 추천 의향']
        }
    },
    
    '분석용_구분변수': [
        '월별', '분기별', '국가별', '국가별(중동)', '성별', '연령별', '방한목적별', '방한횟수별', '여행형태별'
    ]
}


In [23]:
survey_structure

{'기본정보': ['아이디'],
 '여행_준비': {'방한_목적_및_계기': {'주요_방한_목적': ['문1. 주요 방한 목적'],
   '관심_계기': ['문1-1. 한국여행 관심 계기(1순위)',
    '문1-1. 한국여행 관심 계기(2순위)',
    '문1-1. 한국여행 관심 계기(3순위)']},
  '방문국_고려': {'고려한_아시아_국가': ['문2. 방문을 고려한 아시아 국가(1순위)',
    '문2. 방문을 고려한 아시아 국가(2순위)',
    '문2. 방문을 고려한 아시아 국가(3순위)',
    '문2. 방문을 고려한 아시아 국가(없음)'],
   '타국가_관심_계기': ['문2-1. 해당 국가 관심 계기(1순위)',
    '문2-1. 해당 국가 관심 계기(2순위)',
    '문2-1. 해당 국가 관심 계기(3순위)'],
   '연계여행_정보': ['문2-2. 한국만 방문(카테고리)',
    '문2-2. 한국 방문 직전 타 국가 방문(카테고리)',
    '문2-2. 한국 방문 직후 타 국가 방문(카테고리)',
    '문2-2. 한국 방문 전후 타 국가 방문(카테고리)',
    '문2-2. 한국방문 직전 방문국가',
    '문2-2. 한국방문 직후 방문국가',
    '문2-2. 한국방문 직전 방문국가(숙박일수)',
    '문2-2. 한국방문 직후 방문국가(숙박일수)']},
  '고려사항': {'고려한_관광활동': ['문3-1. 고려한 관광활동(1순위)',
    '문3-1. 고려한 관광활동(2순위)',
    '문3-1. 고려한 관광활동(3순위)'],
   '고려한_관광인프라': ['문3-2. 고려한 관광인프라(1순위)',
    '문3-2. 고려한 관광인프라(2순위)',
    '문3-2. 고려한 관광인프라(3순위)']},
  '정보_수집': {'정보_수집_경로': ['문4. 여행 전 한국 관련 정보 수집 경로(1순위)',
    '문4. 여행 전 한국 관련 정보 수집 경로(2순위)',
    '문4. 여행 전 한국 관련

In [24]:
def clean_text(text):
    """문자열에서 특정 특수문자를 제거하고 양쪽 공백을 정리하는 함수"""
    if isinstance(text, str):
        # \xa0 (줄바꿈 없는 공백), ▩ 제거 후 strip()으로 양쪽 공백 제거
        return text.replace('\xa0', '').replace('▩', '').strip()
    return text

def clean_nested_structure(obj):
    """중첩된 딕셔너리나 리스트를 순회하며 모든 문자열을 정리하는 재귀 함수"""
    if isinstance(obj, dict):
        # 딕셔너리일 경우: 각 key와 value에 대해 재귀적으로 함수 호출
        return {clean_text(k): clean_nested_structure(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        # 리스트일 경우: 각 요소에 대해 재귀적으로 함수 호출
        return [clean_nested_structure(elem) for elem in obj]
    else:
        # 문자열이거나 다른 타입일 경우: clean_text 함수만 적용
        return clean_text(obj)

cleaned_survey_structure = clean_nested_structure(survey_structure)

cleaned_survey_structure

{'기본정보': ['아이디'],
 '여행_준비': {'방한_목적_및_계기': {'주요_방한_목적': ['문1. 주요 방한 목적'],
   '관심_계기': ['문1-1. 한국여행 관심 계기(1순위)',
    '문1-1. 한국여행 관심 계기(2순위)',
    '문1-1. 한국여행 관심 계기(3순위)']},
  '방문국_고려': {'고려한_아시아_국가': ['문2. 방문을 고려한 아시아 국가(1순위)',
    '문2. 방문을 고려한 아시아 국가(2순위)',
    '문2. 방문을 고려한 아시아 국가(3순위)',
    '문2. 방문을 고려한 아시아 국가(없음)'],
   '타국가_관심_계기': ['문2-1. 해당 국가 관심 계기(1순위)',
    '문2-1. 해당 국가 관심 계기(2순위)',
    '문2-1. 해당 국가 관심 계기(3순위)'],
   '연계여행_정보': ['문2-2. 한국만 방문(카테고리)',
    '문2-2. 한국 방문 직전 타 국가 방문(카테고리)',
    '문2-2. 한국 방문 직후 타 국가 방문(카테고리)',
    '문2-2. 한국 방문 전후 타 국가 방문(카테고리)',
    '문2-2. 한국방문 직전 방문국가',
    '문2-2. 한국방문 직후 방문국가',
    '문2-2. 한국방문 직전 방문국가(숙박일수)',
    '문2-2. 한국방문 직후 방문국가(숙박일수)']},
  '고려사항': {'고려한_관광활동': ['문3-1. 고려한 관광활동(1순위)',
    '문3-1. 고려한 관광활동(2순위)',
    '문3-1. 고려한 관광활동(3순위)'],
   '고려한_관광인프라': ['문3-2. 고려한 관광인프라(1순위)',
    '문3-2. 고려한 관광인프라(2순위)',
    '문3-2. 고려한 관광인프라(3순위)']},
  '정보_수집': {'정보_수집_경로': ['문4. 여행 전 한국 관련 정보 수집 경로(1순위)',
    '문4. 여행 전 한국 관련 정보 수집 경로(2순위)',
    '문4. 여행 전 한국 관련

In [25]:
import pandas as pd
import re
import json
import numpy as np # float 타입 비교를 위해 numpy import


def process_survey_row(row, structure):
    """
    [최종 수정] 중복 컬럼 문제를 해결하고, 로직을 단순화한 최종 버전
    """
    result_dict = {}

    for key, sub_structure in structure.items():
        # 1. 하위 구조가 딕셔너리면, 재귀 호출로 더 깊이 들어감
        if isinstance(sub_structure, dict):
            nested_result = process_survey_row(row, sub_structure)
            if nested_result:  # 빈 딕셔너리가 아니면 결과에 추가
                result_dict[key] = nested_result
        
        # 2. 하위 구조가 리스트(컬럼명 목록)이면, 실제 값 처리
        elif isinstance(sub_structure, list):
            sub_dict = {}
            for col_name in sub_structure:
                if col_name in row:
                    value = row[col_name]

                    cleaned_key = re.sub(r'^문[\d-]+\.\s*', '', col_name)
                    # 2. key의 양쪽 공백 제거
                    cleaned_key = cleaned_key.strip()

                    # ▼▼▼ [핵심 수정] NaN 값을 '응답 없음'으로 처리하는 로직 ▼▼▼
                    # 값이 Series이면 (중복 컬럼)
                    if isinstance(value, pd.Series):
                        valid_values = value.dropna()
                        if not valid_values.empty:
                            
                            first_value = valid_values.iloc[0]
                            if isinstance(first_value, (float, np.floating)):
                                sub_dict[cleaned_key] = int(round(first_value))
                            else:
                                sub_dict[cleaned_key] = first_value
                        else: # Series의 모든 값이 NaN인 경우
                            sub_dict[cleaned_key] = '응답 없음' 
                    # 단일 값이면
                    else:
                        if pd.notna(value):
                            if isinstance(value, (float, np.floating)):
                                sub_dict[cleaned_key] = int(round(value))
                            else:
                                sub_dict[cleaned_key] = value
                        else: # 단일 값이 NaN인 경우
                            sub_dict[cleaned_key] = '응답 없음'
                    # ▲▲▲ 수정 완료 ▲▲▲

            if sub_dict: # 만들어진 딕셔너리가 비어있지 않으면 결과에 추가
                result_dict[key] = sub_dict
                
    return result_dict

# --- 메인 실행 ---
# 최종 결과를 담을 리스트
all_structured_data = []

# df_processed의 모든 행에 대해 반복 실행
cnt = 0
for index, row in df_processed.iterrows():
    # '분석용_구분변수'는 보통 개인 정보가 아니므로 구조 생성에서 제외
    temp_structure = survey_structure.copy()
    #temp_structure.pop('분석용_구분변수', None) 
    
    structured_entry = process_survey_row(row, temp_structure)
    all_structured_data.append(structured_entry)
    cnt += 1
    if cnt > 100:
        break


# --- 결과 확인 (첫 번째 응답자 데이터) ---
if all_structured_data:
    print(json.dumps(all_structured_data[0], indent=2, ensure_ascii=False))


{
  "기본정보": {
    "아이디": 57
  },
  "여행_준비": {
    "방한_목적_및_계기": {
      "주요_방한_목적": {
        "주요 방한 목적": "여가, 위락, 휴식"
      },
      "관심_계기": {
        "한국여행 관심 계기(1순위)": "한류 콘텐츠(K-pop, 드라마, 영화 등)를 접하고 나서",
        "한국여행 관심 계기(2순위)": "응답 없음",
        "한국여행 관심 계기(3순위)": "응답 없음"
      }
    },
    "방문국_고려": {
      "고려한_아시아_국가": {
        "방문을 고려한 아시아 국가(1순위)": "응답 없음",
        "방문을 고려한 아시아 국가(2순위)": "응답 없음",
        "방문을 고려한 아시아 국가(3순위)": "응답 없음",
        "방문을 고려한 아시아 국가(없음)": "없다"
      },
      "타국가_관심_계기": {
        "해당 국가 관심 계기(1순위)": "응답 없음",
        "해당 국가 관심 계기(2순위)": "응답 없음",
        "해당 국가 관심 계기(3순위)": "응답 없음"
      },
      "연계여행_정보": {
        "한국만 방문(카테고리)": "한국만 방문",
        "한국 방문 직전 타 국가 방문(카테고리)": "응답 없음",
        "한국 방문 직후 타 국가 방문(카테고리)": "응답 없음",
        "한국 방문 전후 타 국가 방문(카테고리)": "응답 없음",
        "한국방문 직전 방문국가": "응답 없음",
        "한국방문 직후 방문국가": "응답 없음",
        "한국방문 직전 방문국가(숙박일수)": "응답 없음",
        "한국방문 직후 방문국가(숙박일수)": "응답 없음"
      }
    },
    "고려사항": {
      "고려한